In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from operator import itemgetter

In [4]:
DB_PATH = "./chroma_db"     # 저장 경로

# 문자열 리스트로 생성
db = Chroma.from_texts(
    ["안녕하세요.", "제 이름은 이인환입니다.", "저는 컴퓨터프로그래밍 강사입니다."],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    persist_directory=DB_PATH,
    collection_name='my_db'
)

In [5]:
retriever = db.as_retriever()

In [6]:
# itemgetter를 단축어로 사용하기

# RunnableParallel과 결합할 때 Python의 itemgetter를 단축어로 사용하여 map에서 데이터를 추출할 수 있습니다.
# itemgetter() 를 사용하여 map에서 특정 키를 추출합니다.

In [9]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [7]:
# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

In [8]:
prompt = ChatPromptTemplate.from_template(template)

In [10]:
output_parser = StrOutputParser()

In [11]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | output_parser
)

In [12]:
chain.invoke({"question": "What is 인환's job", "language": "Korean"})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'인환의 직업은 컴퓨터프로그래밍 강사입니다.'